In [1]:
from bs4 import BeautifulSoup

import requests
import re

import pandas as pd

import pandas as pd  # Library for data manipulation and analysis

from geopy.geocoders import Photon
from geopy.geocoders import Nominatim

In [2]:
def url_to_string(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text)
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [3]:
# base url to scrape
url_base = "https://redes.dh.tamu.edu/index.php/node/"

# create empty dataframe to fill out with org's addresses
df_comp = pd.DataFrame(columns=['id_org', 'Organización','Dirección','Ciudad','Estado','País','Código Postal','Tipo de Iniciativa'])

# 3 ids for testing, must change to higher number
id_orgs = range(680)

### PARTE 1: scraping to get addresses

In [ ]:
#texto_org

In [4]:
for id_org in id_orgs:
    # create url
    url_org = '{}{}'.format(url_base, id_org)
    # scrape
    texto_org = url_to_string(url_org)
    
    # Extract the name of the organization
    org = re.findall("(.*?)\|", texto_org)[0]

    # Extract address information
    # error handling returns empty string if it doesn't find the content
    try:
        direccion = re.findall("Dirección (.*?) Ciudad", texto_org)[0]
    except IndexError:
        direccion = ''
    try:
        ciudad = re.findall("Ciudad (.*?) Estado", texto_org)[0]
    except IndexError:
        ciudad = ''
    try:
        estado = re.findall("Estado (.*?) País", texto_org)[0]
    except IndexError:
        estado = ''
    try:
        pais = re.findall("País (.*?) Código Postal", texto_org)[0]
    except IndexError:
        pais = ''
    try:
        cp = re.findall("Código Postal (\d+)", texto_org)[0]
    except IndexError:
        cp = ''
    try:
        tipo_iniciativa = re.findall("Tipo de Iniciativa (.*?) Tipo de Ayuda", texto_org)[0]
    except IndexError:
        tipo_iniciativa = ''
   

        
    # Create a pandas DataFrame with the extracted information
    df_temp = pd.DataFrame({
        'id_org' : id_org,
        'Organización': [org],
        'Dirección': [direccion],
        'Ciudad': [ciudad],
        'Estado': [estado],
        'País': [pais],
        'Código Postal': [cp],
        'Tipo de Iniciativa' : [tipo_iniciativa]
    })

    df_comp = pd.concat([df_comp, df_temp], ignore_index=True)


df_comp.head()

,id_org,Organización,Dirección,Ciudad,Estado,País,Código Postal,Tipo de Iniciativa
0,0,Page not found,,,,,,
1,1,Otros Dreams en Acción,"José María Marroquí 83 Colonia Centro, Centro,...",Ciudad de México,Ciudad de México,México,06000,Asociación civil
2,2,Dirección de CoDHR,,,,,,
3,3,Comunidad Redes,,,,,,
4,4,Acerca de nosotros,,,,,,


In [5]:
# Create a new DataFrame with non-empty "Dirección" values
directorio_redes = df_comp.loc[df_comp['Dirección'] != '']


In [13]:
directorio_redes.head()
#directorio_redes.shape

,id_org,Organización,Dirección,Ciudad,Estado,País,Código Postal,Tipo de Iniciativa
1,1,Otros Dreams en Acción,"José María Marroquí 83 Colonia Centro, Centro,...",Ciudad de México,Ciudad de México,México,06000,Asociación civil
21,21,"Cáritas Mazatlán, I.A.P (Albergue al peregrino)","Toma de Celaya 86, Col. Francisco Villa, C.P. ...",Mazatlán,Sinaloa,México,82127,Albergue
22,22,Albergue Una Gota en el Océano,"Bahía Magdalena 20, Col. Rincón de Urías, C.P....",Mazatlán,Sinaloa,México,82070,Albergue
23,23,Albergue Ejército de Salvación,C. Gutierrez Nájera #514 Colonia Centro 82000,Mazatlán,Sinaloa,México,82000,Albergue
24,24,CAVI Centro de Atención a la Violencia Intraf...,AVE. EJERCITO MEXICANO 2017 ALTOS COL. PALOS P...,Mazatlán,Sinaloa,México,82010,Asociación civil


In [15]:
#directorio_redes.to_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/1. instituciones/redes/nombre_del_archivo.csv', encoding='utf-8')

### direccion (completa)

Eventualmente podria ser necesario procesar mas la informacion y como se extrae de manera que sea mas facil para los geocodificardores encontrar sus coordenadas

propuestas de procesamiento:

-Identificar y remover palabras repetedias, simbolos que no sean letras y no admitir comas consecutivas


-Talvez sea prudente echarle chatgtp a las extracciones igual que como lo hizo fer con el dir de acnur servicios

In [ ]:
# Create a new column with merged information
directorio_redes['DirecciónCompleta'] = directorio_redes['Dirección'] + ',' + directorio_redes['Ciudad'] + ', ' + directorio_redes['Estado']# + ', ' + directorio_redes['País']

# Optionally, you can strip any leading or trailing commas from the merged address
directorio_redes['DirecciónCompleta'] = directorio_redes['DirecciónCompleta'].str.strip(',')


## Pruebas

In [ ]:
#Aqui podriamos probar con distintas configuraciones a la parte de direccion para ver si podemos mejorar
#la extraccion de coordenadas

#Dirección + Ciudad + Estado + País
dr1 = pd.DataFrame()
dr1['name'] = directorio_redes['Organización']
dr1['direccion'] = directorio_redes['Dirección'] + ',' + directorio_redes['Ciudad'] + ', ' + directorio_redes['Estado']# + ', ' + directorio_redes['País']



In [ ]:
dr1.head()

### PARTE 2: geolocate addresses (solo para pruebas, para editar df finales en notebook geocode)

Para seguir trabajando en esta parte hay que definir que servicio es el mas acertado, escoje algunas filas dificiles del csv directorio_redes. haz un data frame, correlo en photon y revisa que servicio es mas acertado. Googlemaps, nominatim, etc etc.

In [ ]:
from geopy.geocoders import Photon


In [ ]:
#geolocatorphoton
def get_lat_lon(address):
    geolocator = Photon(user_agent="measurements",timeout=10)
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except AttributeError:
        return '', ''

In [ ]:
#subset para probar
indices_seleccionados = [1,2,3,4,5,6,7,8]
subset_df = dr1.iloc[indices_seleccionados]
subset_df.head(10)

In [ ]:
# get latlon for each address
subset_df["latlon"] = subset_df["direccion"].apply(get_lat_lon)

In [ ]:
subset_df.head(10)

## 1. Redes a CSV

In [ ]:
# 1.redes
#este es el df a descargar, seleccionar el que mejor responda a conseguir las coordenadas
redes = pd.DataFrame()

redes['name'] = directorio_redes['Organización']
redes['descripcion'] = None  # Filling all rows with "Nan" (None)
redes['latitud'] = None      # Filling all rows with "Nan" (None)
redes['longitud'] = None     # Filling all rows with "Nan" (None)
redes['coordenadas'] = None  # Filling all rows with "Nan" (None)
redes['tipo'] = 'OSC / ONG'  # All rows should say "OSC / ONG"
redes['direccion'] = directorio_redes['DirecciónCompleta']
redes['bd'] = 'redes'        # All rows should say "redes"


#redes.head(5)

In [ ]:
#dfredes = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/1. instituciones/notebooks_institucion/1.Redes.csv', encoding='utf-8')


In [ ]:
#1. Redes#
#Este crea un nuevo bd y lo manda al folder bd_ubicaciones.
#redes.to_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Instituciones/notebooks_institucion/1.Redes.csv', index=False, encoding='utf-8')


In [ ]:
dfredes.head()

In [ ]:
#vamos a validar las direcciones de redes, hacemos pruebas para ver las coordenadas en tableu, y vemos que pedo.


# paso 1.get coordinates

#funcion para hacer direcciones a coordenadas
def get_coordinates(address):
    if pd.isnull(address): # Check if the address is NaN
        return None, None
    geolocator = Photon(user_agent="pablosthesis", timeout=10)
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except AttributeError:
        return None, None

In [ ]:
# Aplicamos la función para obtener las coordenadas
dfredes['latitud'], dfredes['longitud'] = zip(*dfredes['direccion'].apply(get_coordinates))

# Combinamos latitud y longitud en una nueva columna
dfredes['coordenadas'] = dfredes[['latitud', 'longitud']].apply(tuple, axis=1)


In [ ]:
#Revisar outliers, con latitud sacamos que nos ponga de menor a mayor y buscamos

dfredes_ordenado = dfredes.sort_values(by='latitud', ascending=False)



In [ ]:
dfredes_ordenado.head()

#el primer registro en ascendiente esta fuera de mexico, todos los demas bien.

#el primer registro en desc sale en mexicali, buena señal de que de aqui no hay tantos Problemas con Photon.



In [ ]:
#Nan tratamos al ultimo en excel.
#Descargamos y revalidamos en tableu

dfredes_ordenado.to_csv('/Users/pablouriarte/Documents/1.Redesvalidacion.csv', index=False, encoding='utf-8')
